In [4]:
import pandas as pd
import csv

In [30]:
df = pd.read_csv('data/nyt_ts.csv', index_col='timestamp')
states = df['state'].unique()

In [32]:
# Need to find the growing increments of votes for each time by each state
filt = (df['state'] == 'pennsylvania')
dfpen = df[filt]
dfpen['votes_inc'] = dfpen['votes'].diff(periods=1)
dfpen

,Unnamed: 0,votes,eevp,eevp_source,state,expected_votes,trump2016,votes2012,votes2016,vote_share_rep,vote_share_dem,vote_share_trd,votes_inc
timestamp,,,,,,,,,,,,,
2020-11-04T09:25:23Z,6543,0,0,edison,pennsylvania,6900000,2970733,5742040,6166708,0.000,0.000,0,NaN
2020-11-04T00:19:27Z,6544,77,0,edison,pennsylvania,6900000,2970733,5742040,6166708,0.429,0.571,0,77.0
2020-11-04T00:30:20Z,6545,0,0,edison,pennsylvania,6900000,2970733,5742040,6166708,0.000,0.000,0,-77.0
2020-11-04T00:42:45Z,6546,11,0,edison,pennsylvania,6900000,2970733,5742040,6166708,0.364,0.636,0,11.0
2020-11-04T00:59:15Z,6547,0,0,edison,pennsylvania,6900000,2970733,5742040,6166708,0.000,0.000,0,-11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-10T18:16:47Z,7089,6776099,98,edison,pennsylvania,6900000,2970733,5742040,6166708,0.490,0.497,0,174.0
2020-11-10T18:32:56Z,7090,6776102,98,edison,pennsylvania,6900000,2970733,5742040,6166708,0.490,0.497,0,3.0
2020-11-10T18:38:47Z,7091,6776417,98,edison,pennsylvania,6900000,2970733,5742040,6166708,0.490,0.497,0,315.0


In [36]:
dfg = df.groupby('state')
dfg.last()

,Unnamed: 0,votes,eevp,eevp_source,expected_votes,trump2016,votes2012,votes2016,vote_share_rep,vote_share_dem,vote_share_trd
state,,,,,,,,,,,
alabama,178,2309900,99,edison,2300000,1318255,2074338,2123372,0.621,0.365,0
alaska,4,190970,61,edison,367000,163387,300495,318608,0.622,0.336,0
arizona,419,3335055,98,edison,3400000,1252401,2306559,2604657,0.490,0.494,0
arkansas,317,1209360,99,edison,1225000,684872,1069468,1130635,0.626,0.346,0
california,588,14973644,89,edison,16800000,4483810,13038547,14237884,0.332,0.648,0
colorado,702,3173472,96,edison,3300000,1202484,2569520,2780247,0.421,0.553,0
connecticut,926,1821824,98,edison,1850000,673215,1558993,1644920,0.392,0.592,0
delaware,936,502392,99,edison,510000,185127,413921,443814,0.398,0.588,0
florida,1221,11075446,99,edison,11100000,4617886,8474179,9501617,0.512,0.478,0


In [25]:
# Calculate absolute vote values per time stamp



df['trump_v'] = [round(sh * v) for sh,v in zip(df['vote_share_rep'],df['votes'])]
df['biden_v'] = [round(sh * v) for sh,v in zip(df['vote_share_dem'],df['votes'])]
df['trd_v']= [round(sh * v) for sh,v in zip(df['vote_share_trd'],df['votes'])]
df['check_v'] = [d-(a+b+c) for a,b,c,d in zip(df['trump_v'],df['biden_v'],df['trd_v'],df['votes'])]
# check_v is showing rounding errors

df

,Unnamed: 0,votes,eevp,eevp_source,state,expected_votes,trump2016,votes2012,votes2016,vote_share_rep,vote_share_dem,vote_share_trd,trump_v,biden_v,trd_v,check_v
timestamp,,,,,,,,,,,,,,,,
2020-11-04T10:01:44Z,0,0,0,edison,alaska,367000,163387,300495,318608,0.000,0.000,0,0,0,0,0
2020-11-04T07:47:41Z,1,113744,31,edison,alaska,367000,163387,300495,318608,0.605,0.352,0,68815,40038,0,4891
2020-11-04T08:57:55Z,2,132475,36,edison,alaska,367000,163387,300495,318608,0.614,0.347,0,81340,45969,0,5166
2020-11-04T18:34:58Z,3,172031,56,edison,alaska,367000,163387,300495,318608,0.629,0.330,0,108207,56770,0,7054
2020-11-09T19:09:13Z,4,190970,61,edison,alaska,367000,163387,300495,318608,0.622,0.336,0,118783,64166,0,8021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-04T08:29:35Z,8834,268852,93,edison,wyoming,280000,174419,249061,255849,0.697,0.268,0,187390,72052,0,9410
2020-11-04T08:45:32Z,8835,271429,94,edison,wyoming,280000,174419,249061,255849,0.698,0.267,0,189457,72472,0,9500
2020-11-04T08:50:19Z,8836,276343,95,edison,wyoming,280000,174419,249061,255849,0.699,0.266,0,193164,73507,0,9672


In [27]:
states = df['state'].unique()
filt = (df['state'] == 'illinois')
df[filt]

,Unnamed: 0,votes,eevp,eevp_source,state,expected_votes,trump2016,votes2012,votes2016,vote_share_rep,vote_share_dem,vote_share_trd,trump_v,biden_v,trd_v,check_v
timestamp,,,,,,,,,,,,,,,,
2020-11-04T09:51:24Z,2056,0,0,edison,illinois,6100000,2146015,5242014,5594420,0.000,0.000,0,0,0,0,0
2020-11-04T01:08:08Z,2057,17859,0,edison,illinois,6100000,2146015,5242014,5594420,0.830,0.135,0,14823,2411,0,625
2020-11-04T01:33:59Z,2058,224064,4,edison,illinois,6100000,2146015,5242014,5594420,0.410,0.572,0,91866,128165,0,4033
2020-11-04T01:34:10Z,2059,265204,4,edison,illinois,6100000,2146015,5242014,5594420,0.433,0.547,0,114833,145067,0,5304
2020-11-04T01:36:52Z,2060,319199,5,edison,illinois,6100000,2146015,5242014,5594420,0.424,0.557,0,135340,177794,0,6065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-07T03:26:11Z,2292,5454018,89,edison,illinois,6100000,2146015,5242014,5594420,0.427,0.553,0,2328866,3016072,0,109080
2020-11-08T22:11:54Z,2293,5465440,90,edison,illinois,6100000,2146015,5242014,5594420,0.427,0.554,0,2333743,3027854,0,103843
2020-11-09T13:18:54Z,2294,5501352,90,edison,illinois,6100000,2146015,5242014,5594420,0.426,0.555,0,2343576,3053250,0,104526
